In [13]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [14]:
mutations_ARF9_genotyped = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF9/ARF9_TFBSoverlapping_genotyped.tsv'
mutations_ARF9_genotyped_only_mutated = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF9/ARF9_TFBSoverlapping_genotyped_only_mutated.tsv'
mutations_ARF9_genotyped_only_mutated_flattened = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF9/ARF9_TFBSoverlapping_genotyped_only_mutated_flattened.tsv'

mutations_ARF18_genotyped = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18/ARF18_TFBSoverlapping_genotyped.tsv'
mutations_ARF18_genotyped_only_mutated = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18/ARF18_TFBSoverlapping_genotyped_only_mutated.tsv'
mutations_ARF18_genotyped_only_mutated_filtered = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18/ARF18_TFBSoverlapping_genotyped_only_mutated_filtered.tsv'
mutations_ARF18_genotyped_only_mutated_flattened = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18/ARF18_TFBSoverlapping_genotyped_only_mutated_flattened.tsv'
mutations_ARF18_genotyped_only_mutated_flattened_filtered = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18_filtered/ARF18_TFBSoverlapping_genotyped_only_mutated_flattened.tsv'

mutations_DREB26_genotyped = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/DREB26/DREB26_TFBSoverlapping_genotyped.tsv'
mutations_DREB26_genotyped_only_mutated = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/DREB26/DREB26_TFBSoverlapping_genotyped_only_mutated.tsv'
mutations_DREB26_genotyped_only_mutated_flattened = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/DREB26/DREB26_TFBSoverlapping_genotyped_only_mutated_flattened.tsv'

mutations_NLP7_genotyped = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/NLP7/NLP7_TFBSoverlapping_genotyped.tsv'
mutations_NLP7_genotyped_only_mutated = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/NLP7/NLP7_TFBSoverlapping_genotyped_only_mutated.tsv'
mutations_NLP7_genotyped_only_mutated_flattened = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/NLP7/NLP7_TFBSoverlapping_genotyped_only_mutated_flattened.tsv'


In [15]:
#read in files
def read_files(genotyped_file,genotyped_only_mutated_file,genotyped_only_mutated_flattened_file):
    """read in the files to pandas"""
    #read in files
    genotyped_df = pd.read_table(genotyped_file,sep='\t', header=0)
    genotyped_only_mutated_df = pd.read_table(genotyped_only_mutated_file,sep='\t', header=0)
    genotyped_only_mutated_flattened_df = pd.read_table(genotyped_only_mutated_flattened_file,sep='\t', header=0)
    return genotyped_df,genotyped_only_mutated_df,genotyped_only_mutated_flattened_df


In [16]:
def count_number_mutations_overlapping_at_least_one_TFBS(df, mutation_type):
    """function to count the number of mutations in the df which overlap at least one TFBS"""
    #turn df into a dict
    df_dict = df.to_dict(orient="records")
    mutations_overlapping_at_least_one_TFBS_count = 0
    if mutation_type == 'insertion':
        for row_dict in df_dict:
                list_lists_of_overlapping_AGIs = []
                genomic_positions = literal_eval(row_dict[f'{mutation_type}_genomic_positions_combined'])
                #iterate over insertions. Some insertions are linger than 1 bp so the genomic position pairs need combining
                #remaining pairs
                remaining_pairs = []
                for size in literal_eval(row_dict[f'{mutation_type}_sizes']):
                                
                    size = int(size)
                    #select pairs up to the size of the insertion
                    
                    #pop out the first n pairs from the genomic positions list, where n is the size of the insertion - 1
                    #if remaining pairs not empty, start with that
                    if len(remaining_pairs)>0:
                        #print('not []')
                        current_pairs = remaining_pairs[:size]
                    #else create the remaining pairs variable
                    elif len(remaining_pairs)==0:
                        #print('is []')
                        current_pairs = genomic_positions[:size]
                        remaining_pairs = genomic_positions[size:]          
                    #print(f'currentpairs={current_pairs}remaining={remaining_pairs}')

                    current_pairs_list_of_overlapping = []
                    #iterate over pairs in the mutation_genomic_positions_combined list #evaluate the string representing the pair as if they are literals ie. a list
                    for pair in current_pairs:
                        #print(f'pair={pair}')
                        list_of_AGIs = []
                        list_of_overlapping_AGIs = []
                        #for genomic position in pair 
                        #print(pair)
                        for pos in pair:
                            #make the genomic position string
                            pos = str(pos)
                            #if genomic position is in the mutation_overlapping_TFBS_AGI_combined dictionary
                            if pos in literal_eval(row_dict[f'{mutation_type}_overlapping_TFBS_AGI_combined']):
                                #print(pos)
                                #append to list_of_AGIs
                                mutation_overlapping_TFBS_AGI_combined = literal_eval(row_dict[f'{mutation_type}_overlapping_TFBS_AGI_combined'])
                            
                                #print(pos)
                                AGI = mutation_overlapping_TFBS_AGI_combined[pos]
                                list_of_AGIs += AGI
                        #if empty list, pass
                        
                        for elem in list_of_AGIs:
                            #print(elem)
                            #if TFBS is only overlapping one of the genomic positions in the pair, pass - we only want to include TFBSs that overlapping both positions so that the insertion is totally overlapping
                            if list_of_AGIs.count(elem) == 1:
                                pass
                            elif list_of_AGIs.count(elem) > 1:
                                list_of_overlapping_AGIs.append(elem)
                        #append to list of lists if lists_of_overlapping_AGIs is not empty
                        #remove duplicates from list_of_overlapping_AGIs
                        list_of_overlapping_AGIs_set = set(list_of_overlapping_AGIs)
                        list_of_overlapping_AGIs = list(list_of_overlapping_AGIs_set)
                        if len(list_of_overlapping_AGIs) > 0:
                            #print(list_of_overlapping_AGIs)
                            #get unique AGIs
                            #list_of_overlapping_AGIs_unique
                            current_pairs_list_of_overlapping+=list_of_overlapping_AGIs
                    #get unique elements
                    current_pairs_list_of_overlapping_set = set(current_pairs_list_of_overlapping)
                    current_pairs_list_of_overlapping = list(current_pairs_list_of_overlapping_set)
                    #if current_pairs_list_of_overlapping list isn't empty, append to list_lists_of_overlapping_AGIs 
                    if len(current_pairs_list_of_overlapping) > 0:
                        #print(current_pairs_list_of_overlapping)
                        #add one to the count of mutations which overlap at least one TFBS
                        mutations_overlapping_at_least_one_TFBS_count+=1
                        #append current_pairs_list_of_overlapping of TFBSs to list_lists_of_overlapping_AGIs
                        list_lists_of_overlapping_AGIs.append(current_pairs_list_of_overlapping)
    else:
        for row_dict in df_dict:
            list_lists_of_overlapping_AGIs = []
            genomic_positions = literal_eval(row_dict[f'{mutation_type}_genomic_positions_combined'])
            #iterate over insertions. Some insertions are linger than 1 bp so the genomic position pairs need combining
            #remaining positions
            remaining_positions = []
            for size in literal_eval(row_dict[f'{mutation_type}_sizes']):
                            
                size = int(size)
                #select positions up to the size of the insertion
                
                #pop out the first n positions from the genomic positions list, where n is the size of the insertion - 1
                #if remaining positions not empty, start with that
                if len(remaining_positions)>0:
                    current_positions = remaining_positions[:size]
                    #else create the remaining pairs variable
                elif len(remaining_positions)==0:
                    current_positions = genomic_positions[:size]
                    remaining_positions = genomic_positions[size:]
                list_of_overlapping_AGIs = []
                #for genomic position in pair 
                #print(pair)
                current_positions_list_of_overlapping = []
                for pos in current_positions:
                    #make the genomic position string
                    pos = str(pos)
                    #if genomic position is in the mutation_overlapping_TFBS_AGI_combined dictionary
                    if pos in literal_eval(row_dict[f'{mutation_type}_overlapping_TFBS_AGI_combined']):
                        #print(pos)
                        #append to list_of_overlapping_AGIs
                        mutation_overlapping_TFBS_AGI_combined = literal_eval(row_dict[f'{mutation_type}_overlapping_TFBS_AGI_combined'])
                    
                        #print(pos)
                        AGI = mutation_overlapping_TFBS_AGI_combined[pos]
                        list_of_overlapping_AGIs += AGI                     
                        #append to list of lists if lists_of_overlapping_AGIs is not empty
                #remove duplicates from list_of_overlapping_AGIs
                list_of_overlapping_AGIs_set = set(list_of_overlapping_AGIs)
                list_of_overlapping_AGIs = list(list_of_overlapping_AGIs_set)
                if len(list_of_overlapping_AGIs) > 0:
                    #print(list_of_overlapping_AGIs)
                    #get unique AGIs
                    #list_of_overlapping_AGIs_unique
                    current_positions_list_of_overlapping+=list_of_overlapping_AGIs
                #get unique elements
                current_positions_list_of_overlapping_set = set(current_positions_list_of_overlapping)
                current_positions_list_of_overlapping = list(current_positions_list_of_overlapping_set)
                #if current_positions_list_of_overlapping list isn't empty, append to list_lists_of_overlapping_AGIs 
                if len(current_positions_list_of_overlapping) > 0:
                    #add one to the count of mutations which overlap at least one TFBS
                    mutations_overlapping_at_least_one_TFBS_count+=1
                    #append current_positions_list_of_overlapping of TFBSs to list_lists_of_overlapping_AGIs
                    list_lists_of_overlapping_AGIs.append(current_positions_list_of_overlapping)

    return mutations_overlapping_at_least_one_TFBS_count
                

In [17]:
def count_mutations(genotyped_df,genotyped_only_mutated_df,genotyped_only_mutated_flattened_df):
    """function to count number of genotyped plant lines and also number of plant lines containing mutations, and then number of mutations overall"""
    #count number of unique plant IDs
    genotyped_counts = genotyped_df.plant_ID.unique()
    genotyped_only_mutated_counts = genotyped_only_mutated_df.plant_ID.unique()
    #print counts
    #total mutations
    #total_mutations = genotyped_only_mutated_flattened_df
    print(f'total plant lines sequenced={len(genotyped_counts)} \ntotal mutated plant lines={len(genotyped_only_mutated_counts)}')#\ntotal mutations={len(genotyped_only_mutated_df)}
    #count number of chimeric plant lines
    # unique_line_mutated = genotyped_only_mutated_df.drop_duplicates(subset='plant_ID',keep='first')
    
    unique_line_mutated = genotyped_only_mutated_flattened_df
    non_chimeric = unique_line_mutated[unique_line_mutated.genotype != 'chimeric']
    homozygous = unique_line_mutated[unique_line_mutated.genotype == 'homozygous']
    heterozygous = unique_line_mutated[unique_line_mutated.genotype == 'heterozygous']
    biallelic = unique_line_mutated[unique_line_mutated.genotype == 'biallelic']

    #print bold heading
    print('\033[1m'+'Genotypes'+'\033[0m')
    print(f'number of non-chimeric mutated plant lines = {len(non_chimeric)} ({len(non_chimeric)/len(genotyped_only_mutated_counts)*100}% of non-chimeric lines)')
    print(f'number of homozygous mutated plant lines = {len(homozygous)} ({len(homozygous)/len(non_chimeric)*100}% of non-chimeric lines)')
    print(f'number of heterozygous mutated plant lines = {len(heterozygous)} ({len(heterozygous)/len(non_chimeric)*100}% of non-chimeric lines)')
    print(f'number of biallelic mutated plant lines = {len(biallelic)} ({len(biallelic)/len(non_chimeric)*100}% of non-chimeric lines)')
    #count number of non-chimeric insertions, deletions, substitutions (if two insertions around same guide, counted as once)
    non_chimeric_all = non_chimeric.copy()
    insertions = non_chimeric_all[non_chimeric_all.mutation_type.str.contains('insertion')]
    deletions = non_chimeric_all[non_chimeric_all.mutation_type.str.contains('deletion')]
    substitutions = non_chimeric_all[non_chimeric_all.mutation_type.str.contains('substitution')]
    #number of insertions. evaluate the strings as if they are literals (as if they are lists)
    number_of_insertions = len(insertions.insertion_sizes.apply(literal_eval).sum())
    #print(np.concatenate(insertions.insertion_genomic_positions_combined)
    #number of deletions
    all_deletion_lengths = deletions.deletion_sizes.apply(literal_eval).sum()
    number_of_deletions = len(all_deletion_lengths)
    #number of substitutions
    number_of_substitutions = len(substitutions.substitution_sizes.apply(literal_eval).sum())


    median_reads_guide_sites = genotyped_df.read_number.median()
    median_mutation_reads_guide_sites = genotyped_only_mutated_df.read_number.median()
    #print bold heading
    #filter out deletion values less than 10
    deletions_over_10bp = list(filter(lambda x: x > 10, all_deletion_lengths))
    total_mutations = number_of_insertions + len(deletions_over_10bp) + number_of_substitutions
    print('\033[1m'+'Mutation types'+'\033[0m')
    print(f'total mutations = {total_mutations}')
    print(f'number of insertions = {number_of_insertions} ({number_of_insertions/total_mutations*100}% of total mutations)')
    print(f'number of deletions = {number_of_deletions} ({number_of_deletions/total_mutations*100}% of total mutations)')
    #deletions over 10bp
    #deletions.deletion_positions.str.len()
    #deletions_over_10bp = deletions[deletions.deletion_position.str.len()>10]
    
    #print(all_deletion_lengths)
    
    #print(deletions_over_10bp)
    print(f'deletions over 10bp = {len(deletions_over_10bp)} ({len(deletions_over_10bp)/number_of_deletions*100}% of deletions)')
    print(f'number of substitutions = {number_of_substitutions} ({number_of_substitutions/total_mutations*100}% of total mutations)')
    print(f'median reads at guide sites including wt = {median_reads_guide_sites}')
    print(f'median reads at guide sites mutations only = {median_mutation_reads_guide_sites}')
    
    #find number of mutations overlapping FIMO predicted TFBS
    insertions_overlapping_TFBS = insertions[insertions.insertion_overlapping_TFBS_AGI_combined.notna()]
    deletions_overlapping_TFBS = deletions[deletions.deletion_overlapping_TFBS_AGI_combined.notna()]
    substitutions_overlapping_TFBS = substitutions[substitutions.substitution_overlapping_TFBS_AGI_combined.notna()]
    #count number of insertions overlapping TFBSs
    insertions_count = count_number_mutations_overlapping_at_least_one_TFBS(insertions_overlapping_TFBS, 'insertion')
    deletions_count = count_number_mutations_overlapping_at_least_one_TFBS(deletions_overlapping_TFBS, 'deletion')
    substitutions_count = count_number_mutations_overlapping_at_least_one_TFBS(substitutions_overlapping_TFBS, 'substitution')
          

    #print bold heading
    print('\033[1m'+'Overlapping TFBSs'+'\033[0m')
    print(f'number of insertions overlapping TFBS = {insertions_count} ({insertions_count/number_of_insertions*100}% of insertions)')
    print(f'number of deletions overlapping TFBS = {deletions_count} ({deletions_count/number_of_deletions*100}% of insertions)')
    print(f'number of substitutions overlapping TFBS = {substitutions_count} ({substitutions_count/number_of_substitutions*100}% of insertions)')

    #how many within 7bp of cutsite
    #insertions_7bp = insertions[[[i for i in x if int(i) <8] for x in insertions.insertion_cut_site_distance.str.strip('][').str.split(', ')]]

   # print(insertions_7bp)


    return genotyped_counts, genotyped_only_mutated_counts, genotyped_only_mutated_df

In [18]:
ARF9_genotyped_df, ARF9_mutated_df,ARF9_flattened_df = read_files(mutations_ARF9_genotyped,mutations_ARF9_genotyped_only_mutated,mutations_ARF9_genotyped_only_mutated_flattened)


In [19]:
ARF9_genotyped_plant_lines, ARF9_mutated_plant_lines,ARF9_mutations = count_mutations(ARF9_genotyped_df,ARF9_mutated_df,ARF9_flattened_df)

total plant lines sequenced=468 
total mutated plant lines=147
Genotypes
number of non-chimeric mutated plant lines = 142 (96.5986394557823% of non-chimeric lines)
number of homozygous mutated plant lines = 92 (64.7887323943662% of non-chimeric lines)
number of heterozygous mutated plant lines = 12 (8.450704225352112% of non-chimeric lines)
number of biallelic mutated plant lines = 38 (26.76056338028169% of non-chimeric lines)
Mutation types
total mutations = 195
number of insertions = 182 (93.33333333333333% of total mutations)
number of deletions = 50 (25.64102564102564% of total mutations)
deletions over 10bp = 9 (18.0% of deletions)
number of substitutions = 4 (2.051282051282051% of total mutations)
median reads at guide sites including wt = 361.0
median reads at guide sites mutations only = 234.0
Overlapping TFBSs
number of insertions overlapping TFBS = 34 (18.681318681318682% of insertions)
number of deletions overlapping TFBS = 15 (30.0% of insertions)
number of substitutions ov

In [20]:
#filter ARF18 guide 14 since very repetitive and lots of sequencing errors
ARF18_mutated_df = pd.read_table(mutations_ARF18_genotyped_only_mutated,sep='\t', header=0)
ARF18_mutated_plant_lines_filtered = ARF18_mutated_df[ARF18_mutated_df.guide != 'ARF18_guide14']
#save df
ARF18_mutated_plant_lines_filtered.to_csv(mutations_ARF18_genotyped_only_mutated_filtered, sep='\t',header=True)
#Then I ran pacbio_analyse_flattened_mutations.py on the filtered .tsv to generate the mutations_ARF18_genotyped_only_mutated_flattened_filtered file (f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18_filtered/ARF18_TFBSoverlapping_genotyped_only_mutated_flattened.tsv')


In [21]:
ARF18_genotyped_df, ARF18_mutated_df,ARF18_flattened_filtered_df = read_files(mutations_ARF18_genotyped,mutations_ARF18_genotyped_only_mutated,mutations_ARF18_genotyped_only_mutated_flattened_filtered)

#filter ARF18 guide 14 since very repetitive and lots of sequencing errors
ARF18_genotyped_only_mutated_df = pd.read_table(mutations_ARF18_genotyped_only_mutated,sep='\t', header=0)
ARF18_mutated_plant_lines_filtered = ARF18_mutated_df[ARF18_mutated_df.guide != 'ARF18_guide14']
#save as df

In [22]:

#count mutations
ARF18_genotyped_plant_lines, ARF18_mutated_plant_lines,ARF18_mutations = count_mutations(ARF18_genotyped_df,ARF18_mutated_plant_lines_filtered,ARF18_flattened_filtered_df)

total plant lines sequenced=373 
total mutated plant lines=67
Genotypes
number of non-chimeric mutated plant lines = 66 (98.50746268656717% of non-chimeric lines)
number of homozygous mutated plant lines = 27 (40.909090909090914% of non-chimeric lines)
number of heterozygous mutated plant lines = 25 (37.878787878787875% of non-chimeric lines)
number of biallelic mutated plant lines = 14 (21.21212121212121% of non-chimeric lines)
Mutation types
total mutations = 98
number of insertions = 87 (88.77551020408163% of total mutations)
number of deletions = 11 (11.224489795918368% of total mutations)
deletions over 10bp = 3 (27.27272727272727% of deletions)
number of substitutions = 8 (8.16326530612245% of total mutations)
median reads at guide sites including wt = 623.0
median reads at guide sites mutations only = 310.5
Overlapping TFBSs
number of insertions overlapping TFBS = 53 (60.91954022988506% of insertions)
number of deletions overlapping TFBS = 7 (63.63636363636363% of insertions)
nu

In [23]:
DREB26_genotyped_df, DREB26_mutated_df,DREB26_flattened_df = read_files(mutations_DREB26_genotyped,mutations_DREB26_genotyped_only_mutated,mutations_DREB26_genotyped_only_mutated_flattened)
DREB26_genotyped_plant_lines, DREB26_mutated_plant_lines,DREB26_mutations = count_mutations(DREB26_genotyped_df,DREB26_mutated_df,DREB26_flattened_df)

total plant lines sequenced=366 
total mutated plant lines=43
Genotypes
number of non-chimeric mutated plant lines = 40 (93.02325581395348% of non-chimeric lines)
number of homozygous mutated plant lines = 15 (37.5% of non-chimeric lines)
number of heterozygous mutated plant lines = 19 (47.5% of non-chimeric lines)
number of biallelic mutated plant lines = 6 (15.0% of non-chimeric lines)
Mutation types
total mutations = 53
number of insertions = 40 (75.47169811320755% of total mutations)
number of deletions = 22 (41.509433962264154% of total mutations)
deletions over 10bp = 2 (9.090909090909092% of deletions)
number of substitutions = 11 (20.754716981132077% of total mutations)
median reads at guide sites including wt = 344.0
median reads at guide sites mutations only = 113.0
Overlapping TFBSs
number of insertions overlapping TFBS = 18 (45.0% of insertions)
number of deletions overlapping TFBS = 16 (72.72727272727273% of insertions)
number of substitutions overlapping TFBS = 2 (18.1818

In [24]:
NLP7_genotyped_df, NLP7_mutated_df,NLP7_flattened_df = read_files(mutations_NLP7_genotyped,mutations_NLP7_genotyped_only_mutated,mutations_NLP7_genotyped_only_mutated_flattened)
NLP7_genotyped_plant_lines, NLP7_mutated_plant_lines,NLP7_mutations = count_mutations(NLP7_genotyped_df,NLP7_mutated_df,NLP7_flattened_df)

total plant lines sequenced=478 
total mutated plant lines=112
Genotypes
number of non-chimeric mutated plant lines = 107 (95.53571428571429% of non-chimeric lines)
number of homozygous mutated plant lines = 53 (49.532710280373834% of non-chimeric lines)
number of heterozygous mutated plant lines = 32 (29.906542056074763% of non-chimeric lines)
number of biallelic mutated plant lines = 22 (20.5607476635514% of non-chimeric lines)
Mutation types
total mutations = 151
number of insertions = 127 (84.10596026490066% of total mutations)
number of deletions = 39 (25.82781456953642% of total mutations)
deletions over 10bp = 10 (25.64102564102564% of deletions)
number of substitutions = 14 (9.271523178807946% of total mutations)
median reads at guide sites including wt = 646.0
median reads at guide sites mutations only = 349.0
Overlapping TFBSs
number of insertions overlapping TFBS = 41 (32.28346456692913% of insertions)
number of deletions overlapping TFBS = 7 (17.94871794871795% of insertion